## Notebook to classify snow-covered area (SCA) in PlanetScope 4-band imagery

Rainey Aberle

2022

### Import packages

In [ ]:
import os
import glob
import numpy as np
import geopandas as gpd
import rasterio as rio
from rasterio.mask import mask
from rasterio.plot import show
import earthpy.spatial as es
import matplotlib
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import matplotlib.pyplot as plt
from shapely.geometry import shape
import pickle
import sys
import time

### Define settings and paths in directory

In [ ]:
# -----Determine whether to save output figures
save_figures = True # = True to save output figures
crop_to_AOI = True # = True to crop images to AOI before calculating SCA

# -----Define desired EPSG
epsg = 32606

# -----Define paths in directory
# base directory (path to "planet-snow/")
base_path = '/Users/raineyaberle/Research/PhD/planet-snow/'
# image directory
im_path = base_path+'../study-sites/Wolverine/imagery/Planet/2021-04-20_2021-08-25/adjusted-radiometry/'
# figures output directory
figures_out_path = base_path+'../study-sites/Wolverine/figures/SCA/'
# AOI shapefile full path 
AOI_fn = base_path+'../GIS_data/wolverine_RGI.shp'

### Load files from directory

- AOI
- PlanetScope images
- Image classifier

In [ ]:
# -----Load Area of Interest (AOI) for image cropping
AOI = gpd.read_file(AOI_fn)
# Reproject to imagery CRS if necessary
AOI = AOI.to_crs(epsg)

# -----Load image names from file
os.chdir(im_path) # change directory
im_names = glob.glob('*.tif') # load all .tif file names
im_names.sort() # sort file names by date

# -----Load image classifier
clf_fn = base_path+'inputs-outputs/best_classifier.sav'
clf = pickle.load(open(clf_fn, 'rb'))

# -----Add path to functions
sys.path.insert(1, base_path+'functions/')
from calculate_SCA import crop_images_to_AOI, classify_image, calculate_SCA

### Classify snow and calculate SCA in each image

In [ ]:
# Initialize image dates and SCA
im_dts = [] # image datetimes
SCA = [] # [m^2]
# crop images if previously selected
if crop_to_AOI==True:
    
    # Crop images if previously selected
    cropped_im_path = crop_images_to_AOI(im_path, im_names, AOI)
    # grab cropped image names
    os.chdir(cropped_im_path) # change directory
    im_names_crop = glob.glob('*_crop.tif')
    im_names_crop.sort() # sort file names by date

    # loop through cropped images
    for im_name in im_names_crop:

        # extract datetime from image name
        im_dt = np.datetime64(im_name[0:4] + '-' + im_name[4:6] + '-' + im_name[6:8]
                              + ' ' + im_name[9:11] + ':' +im_name[11:13] + ':' + im_name[13:15])
        im_dts = im_dts + [im_dt]

        # open image
        im = rio.open(im_name)

        # classify snow
        im_x, im_y, snow, fig = classify_image(im, clf)
        fig.suptitle(im_dt)
        plt.show()
        
        # calculate SCA [m^2]
        SCA = SCA + [calculate_SCA(im, snow)]

        # save figure
        if save_figures==True:
            fig.savefig(figures_out_path+im_name[0:15]+'_SCA.png', dpi=200, facecolor='white', edgecolor='none')
            print('figure saved to file')
        
